### Notebook summary
In this notebook we have trained a model with featurs by using domain knowledge from the research by [Zhong and Hitchcock (2021)](https://github.com/Shanlearning/SP-500-Stock-Prediction/tree/master) for AAPL ticker using SPO framework.
Below points were observed:
- Compared to previous experiments loss is reduced significantly
- Hyper parameter tuning yielded good results
- Compared with MAE, MSE and Huber loss where MSE and Huber loss performs better
- Shapley interpretability hows clear distinction between SPO framework's decision making and decision making of other loss functions

Next steps:
- Improvement in scalability with deep learning
- Rigorous testing of SPO framework on various cases in finance
- Inclusion of unknown parameters in SPO framework

### Import libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os
import itertools
from concurrent.futures import ThreadPoolExecutor
import math
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import random
import gurobipy as gp
from gurobipy import GRB
import tensorflow as tf
from tensorflow.keras import initializers
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
import yaml
from pathlib import Path
import shap

### Import modules

In [ ]:
import sys
sys.path.append("../src")

import data_exploration as de
import model_training as mt

### Load necessary directories

In [ ]:
current_dir = Path(os.getcwd())
root_dir = current_dir
while 'Portfolio Optimization using SPO' in root_dir.parts:
    root_dir = root_dir.parent
    if root_dir == Path(root_dir.root):
        print("Root directory not found.")
        break

In [ ]:
config_path = root_dir / "Portfolio Optimization using SPO" / "config" / "config.yml"
complete_data_path = root_dir / "Portfolio Optimization using SPO" / "data" / "dat_518_companies.csv"
data_path = root_dir / "Portfolio Optimization using SPO" / "data" / "AAPL_df.csv"
cost_mat_path = root_dir / "Portfolio Optimization using SPO" / "data" / "cost_mat.csv"
sigma_path = root_dir / "Portfolio Optimization using SPO" / "data" / "sigma_df.csv"

In [ ]:
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

### Import data

In [ ]:
# import data
df_AAPL_train_test = pd.read_csv(data_path)
df_final_returns = pd.read_csv(cost_mat_path)
sigma_df = pd.read_csv(sigma_path)

In [ ]:
gamma = config["gamma"]
sigma = sigma_df.values

### Selecting best features based on domain knowledge

In [ ]:
df_paper_cols = df_AAPL_train_test[config["best_feats"]]

### Split data into train and test

In [ ]:
# training dataframe
df_paper_cols_train, df_paper_cols_test = train_test_split(df_paper_cols, test_size=0.2, random_state=42, 
                                                           shuffle=False)

# cost vector
df_final_returns_train, df_final_returns_test = train_test_split(df_final_returns, test_size=0.2, random_state=42, 
                                                                 shuffle=False)

### Initialize the model

In [ ]:
ppr_n_rows, ppr_n_cols = df_paper_cols_train.shape
ppr_n_feats = ppr_n_cols-1

# Instantiate the model
spo_model_ppr_data_2 = mt.get_model(n_feats = ppr_n_feats)
spo_model_ppr_data_2.summary()

### Grid search

In [ ]:
%%time
# Grid Search to find best hyper-parameters

# Parameters
grid_params_df_paper = config["snp_grid_params"]

# Searching Best Parameters
df_paper_results, df_paper_best_params, df_paper_error = mt.grid_search(df_paper_cols_train, df_final_returns_train, sigma=sigma, gamma=gamma, n_epoch=200, GridSearchParams=grid_params_df_paper)

# Print results
print("Results:")
for res in df_paper_results:
    print(res)

print("Best Parameters:", df_paper_best_params)
print("Error:", df_paper_error)

### Train the model
We will train the model with best hyper-parameters

In [ ]:
%%time
trained_ppr_model_2, epoch_ppr_loss_list_2 = mt.SGD_regressor(df_paper_cols_train, spo_model_ppr_data_2, df_final_returns_train, sigma, gamma, learning_rate= 0.00001, decay_rate=2.02, n_epochs=200, batch_size = 512, decay = 1)

### Plot loss progression with every epoch

In [ ]:
fig_ppr_spo = px.line(epoch_ppr_loss_list_2).update_layout(title="Training Loss progression", xaxis_title="epochs", yaxis_title="SPO+ loss",legend={
            "title": "Loss Value"})
fig_ppr_spo.show()

### Testing the model on test data

In [ ]:
y_pred_ppr = trained_ppr_model_2(df_paper_cols_test.iloc[:,0:ppr_n_feats].values)
ppr_spo_test_loss = mt.get_SPO_plus_testing_loss(df_paper_cols_train, df_final_returns_test, y_pred_ppr, sigma=sigma, gamma=gamma)

print(f'The SPO+ loss on testing data is {ppr_spo_test_loss}')

In [ ]:
# Save the entire model as a `.keras` zip archive.
model_save_path = root_dir / "Portfolio Optimization using SPO" / "models" / "trained_spo_model.keras"
trained_ppr_model_2.compile()
trained_ppr_model_2.save(model_save_path, save_format='tf')

### Comparison with models trained on MAE, MSE and Huber loss

### Model trained on MAE loss

In [ ]:
# Instantiate the model
model_ppr_data_mae = mt.get_model(n_feats = ppr_n_feats, use_bias=False)
model_ppr_data_mae.summary()

In [ ]:
model_ppr_data_mae.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.00001),
    loss='mean_absolute_error')

In [ ]:
%%time
history_mae = model_ppr_data_mae.fit(
    df_paper_cols_train[config["comp_vars"]],
    df_paper_cols_train[config["comp_target"]],
    epochs=200,
    batch_size=512,
    # Suppress logging.
    verbose=1)

In [ ]:
y_pred_mae = tf.convert_to_tensor(model_ppr_data_mae.predict(df_paper_cols_test.iloc[:,0:ppr_n_feats].values))
mae_spo_test_loss = mt.get_SPO_plus_testing_loss(df_paper_cols_train, df_final_returns_test, y_pred_mae, sigma=sigma, gamma=gamma)

print(f'The SPO+ loss on testing data is {mae_spo_test_loss}')

### Model trained on MSE loss

In [ ]:
# Instantiate the model
model_ppr_data_mse = mt.get_model(n_feats = ppr_n_feats, use_bias=False)
model_ppr_data_mse.summary()

In [ ]:
model_ppr_data_mse.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.00001),
    loss='mean_squared_error')

In [ ]:
%%time
history_mse = model_ppr_data_mse.fit(
    df_paper_cols_train[config["comp_vars"]],
    df_paper_cols_train[config["comp_target"]],
    epochs=200,
    batch_size=512,
    # Suppress logging.
    verbose=1)

In [ ]:
y_pred_mse = tf.convert_to_tensor(model_ppr_data_mse.predict(df_paper_cols_test.iloc[:,0:ppr_n_feats].values))
mse_spo_test_loss = mt.get_SPO_plus_testing_loss(df_paper_cols_train, df_final_returns_test, y_pred_mse, sigma=sigma, gamma=gamma)

print(f'The SPO+ loss on testing data is {mse_spo_test_loss}')

### Model trained on Huber loss

In [ ]:
# Instantiate the model
model_ppr_data_huber = mt.get_model(n_feats = ppr_n_feats, use_bias=False)
model_ppr_data_huber.summary()

In [ ]:
model_ppr_data_huber.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.00001),
    loss='huber')

In [ ]:
%%time
history_huber = model_ppr_data_huber.fit(
    df_paper_cols_train[config["comp_vars"]],
    df_paper_cols_train[config["comp_target"]],
    epochs=200,
    batch_size=512,
    # Suppress logging.
    verbose=1)

In [ ]:
y_pred_huber = tf.convert_to_tensor(model_ppr_data_huber.predict(df_paper_cols_test.iloc[:,0:ppr_n_feats].values))
huber_spo_test_loss = mt.get_SPO_plus_testing_loss(df_paper_cols_train, df_final_returns_test, y_pred_huber, sigma=sigma, gamma=gamma)

print(f'The SPO+ loss on testing data is {huber_spo_test_loss}')

### Shapley Interpretability

In [ ]:
# For SPO model

# DeepExplainer to explain predictions of the model
explainer_spo = shap.DeepExplainer(trained_ppr_model_2, df_paper_cols_train.iloc[:,0:ppr_n_feats].values)
# compute shap values
shap_values_spo = explainer.shap_values(df_paper_cols_test.iloc[:,0:ppr_n_feats].values)

In [ ]:
# For model trained on MAE

# DeepExplainer to explain predictions of the model
explainer_mae = shap.DeepExplainer(model_ppr_data_mae, df_paper_cols_train.iloc[:,0:ppr_n_feats].values)
# compute shap values
shap_values_mae = explainer_mae.shap_values(df_paper_cols_test.iloc[:,0:ppr_n_feats].values)

In [ ]:
# For model trained on MSE

# DeepExplainer to explain predictions of the model
explainer_MSE = shap.DeepExplainer(model_ppr_data_mse, df_paper_cols_train.iloc[:,0:ppr_n_feats].values)
# compute shap values
shap_values_MSE = explainer_MSE.shap_values(df_paper_cols_test.iloc[:,0:ppr_n_feats].values)

In [ ]:
# For model trained on Huber loss

# DeepExplainer to explain predictions of the model
explainer_huber = shap.DeepExplainer(model_ppr_data_huber, df_paper_cols_train.iloc[:,0:ppr_n_feats].values)
# compute shap values
shap_values_huber = explainer_huber.shap_values(df_paper_cols_test.iloc[:,0:ppr_n_feats].values)

### Shap summary plot

In [ ]:
shap.summary_plot(shap_values_spo[0], plot_type = 'bar', feature_names = df_paper_cols_test.iloc[:,0:ppr_n_feats].columns, show=False)

In [ ]:
shap.summary_plot(shap_values_mae[0], plot_type = 'bar', feature_names = df_paper_cols_test.iloc[:,0:ppr_n_feats].columns)

In [ ]:
shap.summary_plot(shap_values_MSE[0], plot_type = 'bar', feature_names = df_paper_cols_test.iloc[:,0:ppr_n_feats].columns)

In [ ]:
shap.summary_plot(shap_values_huber[0], plot_type = 'bar', feature_names = df_paper_cols_test.iloc[:,0:ppr_n_feats].columns)

### Shap waterfall plot for first observation on for model trained on SPO framework

In [ ]:
shap.plots._waterfall.waterfall_legacy(explainer_spo.expected_value[0].numpy(), shap_values_spo[0][0], feature_names = df_paper_cols_test.iloc[:,0:ppr_n_feats].columns)

After training the model on features selected using the research [Zhong and Hitchcock (2021)](https://github.com/Shanlearning/SP-500-Stock-Prediction/tree/master) The loss and variability is reduced sinificantly and the model seems to be converging. Comparing model trained on SPO framework with other models trained on MAE, MSE and Huber loss, we can see that the SPO model is almost as good as Huber and MSE model in terms of reducing the loss, but a clear distinction in the decision making can be seen from shap values.